In [1]:
import initialize

In [ ]:

# 1) create the optimization algorithm
# 2) decide on th


# x*1 + y* 2
# lambda x, y: x * 1 + y * 2
# Constaint = LTE(x=100, y=50) & GTE(x=0, y=0)
# genetic = Genetic()
# fitter.fit(func, constraint, x=20, y=30, const=['x'], penalty=-inf)
# fitter.step( state=state)
# 1) create the individual -> only x
# 2) populate
# 3) If not first
#    get children
#    cross over
#    elitism
# 3) mutate
# 4) assess
# 5) yield assessments

In [ ]:
from zenkai import tansaku
import zenkai
import torch

x = 20
y = 30


objective = zenkai.FuncObjective(
    lambda x, y: 2 * x + 2.5 * y 
)

# how to convert to float
individual = zenkai.Individual(x=20.0, y=30.0)
population = individual.populate(10)

mutator = tansaku.GaussianNoiser(0.2, 0.0)
population = mutator(population)


population = zenkai.Population(
    x=population['x'].clamp(0, 30),
    y=population['y'].clamp(0, 20)
)


print(population['x'])

# # Add a "Lambda" constraint
# LC(['x', 't'], lambda x, t: 2 * x + 3 * t < 4)
# LC() + 

objective = zenkai.FuncObjective(lambda x, y: 3 * x + 5 * y, penalty=1000)
assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

population = assessor(population)

divider = tansaku.ProbDivider(6)
parents1, parents2 = divider(population)

print(parents1['x'], parents2['x'])
crossover = tansaku.SmoothCrossOver()
children =crossover(parents1, parents2)
print(children['x'])

elitism = tansaku.KBestElitism(2)
children2 = elitism(population, children)
print(children2['x'])


In [ ]:
import typing
from zenkai import tansaku
from zenkai.kaku import Assessment, State
import torch
from functools import partial
import zenkai

class GeneticAlgorithm(zenkai.Itadaki):

    def __init__(self) -> None:
        super().__init__()
        self.divider = tansaku.ProbDivider(8)
        self.elitism = tansaku.KBestElitism(2)
        self.crossover = tansaku.SmoothCrossOver()
        self.mutator = tansaku.GaussianNoiser(0.5, 0.0)
        self.n = 10
        self.iterations = 100

    def optim_iter(self, objective: zenkai.Objective, state: State = None, **kwargs) -> typing.Iterator[Assessment]:
        
        state = state or State()
        my_state = state.mine(self, objective)
        assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

        i = my_state.get_or_set('i', 0)
        if i == 0:
            individual = zenkai.Individual(**kwargs)
            population = individual.populate(self.n)
            population = self.mutator(population)
        else:
            population = my_state.get('population')

        while i < self.iterations:
            
            if i > 0:
                parents1, parents2 = self.divider(population, state)
                
                children = self.crossover(parents1, parents2)
                children = self.mutator(children)
                population = self.elitism(population, children)
            
            population = population.apply(partial(torch.clamp, min=0))

            population = assessor(population)
            my_state['population'] = population
            yield population.stack_assessments()
            i += 1


objective = zenkai.FuncObjective(
    lambda x, y: torch.abs(2 * x + 3 * y), constraint=zenkai.LTE(y=5, x=10) + zenkai.GTE(y=0, x=0), penalty=0, maximize=True
)


genetic_algorithm = GeneticAlgorithm()

for assessment in genetic_algorithm.optim_iter(objective, x=3.0, y=4.0):
    print(assessment)

    

In [ ]:
import typing
from zenkai import tansaku
import zenkai
from zenkai.kaku import Assessment, State
import torch
from functools import partial

class GeneticAlgorithm(zenkai.Itadaki):

    def __init__(self) -> None:
        super().__init__()
        self.divider = tansaku.ProbDivider(8, 2)
        self.elitism = tansaku.KBestElitism(2, 2)
        self.crossover = tansaku.SmoothCrossOver()
        self.mutator = tansaku.GaussianNoiser(0., 0.0)
        self.n = 10
        self.iterations = 100

    def optim_iter(self, objective: zenkai.Objective, state: State = None, **kwargs) -> typing.Iterator[Assessment]:
        
        state = state or State()
        my_state = state.mine(self, objective)
        assessor = tansaku.ObjectivePopAssessor(objective, ['w', 'b'], reduce_from=2)

        i = my_state.get_or_set('i', 0)
        if i == 0:
            individual = zenkai.Individual(**kwargs)
            population = individual.populate(self.n)
            population = self.mutator(population)
        else:
            population = my_state.get('population')

        while i < self.iterations:
            
            if i > 0:
                parents1, parents2 = self.divider(population, state)
                
                children = self.crossover(parents1, parents2)
                children = self.mutator(children)
                population = self.elitism(population, children)
            
            population = assessor(population)
            my_state['population'] = population
            yield population.stack_assessments()
            i += 1


import torch.nn as nn
from zenkai import ThLoss, IO

base_linear = nn.Linear(32, 8)
base_network = nn.Sequential(
    base_linear,
    nn.ReLU()
)

optimize_linear = nn.Linear(32, 8)
optimize_network = nn.Sequential(
    optimize_linear,
    nn.ReLU()
)

x = torch.rand(128, 32)
t = base_network(x)

objective = zenkai.NNLinearObjective(
    optimize_linear, optimize_network, ThLoss('MSELoss'), IO(x), IO(t)
)

# objective = tansaku.FuncObjective(
#     lambda x, y: torch.abs(2 * x + 3 * y), constraint=tansaku.LTE(y=5, x=10) + tansaku.GTE(y=0, x=0), penalty=0, maximize=True
# )


genetic_algorithm = GeneticAlgorithm()

for assessment in genetic_algorithm.optim_iter(objective, w=optimize_linear.weight, b=optimize_linear.bias):
    print(assessment)

    

In [2]:
from zenkai.kikai.experimental import genetic
from torchvision.datasets import MNIST
from torchvision import transforms
import sensei
import torch

learner = genetic.GeneticNetwork()
learner.to('cuda')
transform = transforms.Compose(
    [transforms.ToTensor()]
)
training_dataset = MNIST('~/development/datasets/', True, transform, download=True)
testing_dataset = MNIST('~/development/datasets/', False, transform, download=True)
torch.device("cuda")

record = sensei.train(
    learner, sensei.DLMaterial.load(training_dataset, 128, True),  
    sensei.DLMaterial.load(testing_dataset, 10000, True), 100, use_io=True
)

100%|██████████| 1/1 [00:00<00:00,  1.15it/s, loss=0.221, Epoch=0/1]


KeyError: 'x'